# **Challenge Overview – Stock Agent**

Acting as consultants for an investment fund managing a portfolio of 11 stocks, your goal is to build an AI-powered solution that enhances investment decision-making. You may focus on the full portfolio or a specific subset. The aim is to demonstrate how AI can drive financial insights, whether through a trading agent, an analytical dashboard, or another innovative tool.
<br>
<br>
### **Tech & Tools**

It is mandatory to develop the solution in **Google Colab** using **Python**.

Other than that, you are completely free to choose your own:

•	Libraries and packages: Use any tool you need (e.g., Pandas, Scikit-learn, LangChain, etc.)

•	Visualization tools: Python-based tools (Matplotlib, Seaborn), Power BI, Tableau, etc. (if you use external visualization tools, don't forget to include prints in the submission zip folder)

•	AI assistants: Feel free to consult ChatGPT, GitHub Copilot, Gemini, or any other.
<br>
<br>
### **Tech Configuration**

**1** - The second code cell contains the code needed to export the dataset for the 11 assets. All data is saved as individual .csv files in a data/ directory, named according to the asset and frequency (e.g., AMZN_hourly.csv or AMZN_daily.csv).

**2** - The remaining cells include functions suggested by the dev team to help accelerate your solution. Each function comes with a description of its purpose and examples of expected usage. Feel free to use, adapt, extend, or completely rework them to fit your approach.

In [ ]:
!pip install yfinance==0.2.59 pandas==2.0.3 matplotlib==3.7.5 seaborn==0.13.2 -q

ERROR: Could not find a version that satisfies the requirement pandas==2.2.2 (from versions: 0.1, 0.2, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.5.0, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.19.2, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.21.0, 0.21.1, 0.22.0, 0.23.0, 0.23.1, 0.23.2, 0.23.3, 0.23.4, 0.24.0, 0.24.1, 0.24.2, 0.25.0, 0.25.1, 0.25.2, 0.25.3, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.1.0, 1.1.1, 1.1.2, 1.1.3, 1.1.4, 1.1.5, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.2.4, 1.2.5, 1.3.0, 1.3.1, 1.3.2, 1.3.3, 1.3.4, 1.3.5, 1.4.0rc0, 1.4.0, 1.4.1, 1.4.2, 1.4.3, 1.4.4, 1.5.0rc0, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 2.0.0rc0, 2.0.0rc1, 2.0.0, 2.0.1, 2.0.2, 2.0.3)
ERROR: No matching distribution found for pandas==2.2.2


**Data Fetching**

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import os

# List of symbols to download
symbols = ["AMZN", "AAPL", "GOOGL", "MSFT", "UDMY", "NXE", "SPY",
           "CDR.WA", "EH", "BTC-USD", "ETH-USD"]

# Set date range
end_date = datetime.now()
start_date = end_date - timedelta(days=729)

start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

print(f"Downloading hourly data from {start_str} to {end_str} (729 days)")

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Download data for each symbol
for symbol in symbols:
    print(f"\nDownloading hourly data for {symbol}...")

    try:
        # Download hourly data
        data = yf.download(symbol, start=start_str, end=end_str, interval="1h", progress=False)

        if not data.empty:
            # Reset index to make Datetime a column
            data.reset_index(inplace=True)

            # Save to CSV
            safe_name = symbol.replace('-', '_').replace('.', '_')
            filename = f"data/{safe_name}_hourly.csv"
            data.to_csv(filename, index=False)
            print(f"Successfully downloaded {len(data)} rows of hourly data for {symbol}")

            # Display the first 5 rows of data
            print(f"\nFirst 5 rows of {symbol} data:")
            print(data.head())

        else:
            print(f"No hourly data available for {symbol}")

            # Try daily data instead
            print(f"Attempting to download daily data for {symbol} instead...")
            daily_data = yf.download(symbol, start=(start_date - timedelta(days=365)).strftime('%Y-%m-%d'),
                                    end=end_str, interval="1d", progress=False)

            if not daily_data.empty:
                daily_data.reset_index(inplace=True)
                filename = f"data/{symbol.replace('-', '_').replace('.', '_')}_daily.csv"
                daily_data.to_csv(filename, index=False)
                print(f"Successfully downloaded {len(daily_data)} rows of daily data for {symbol}")

                # Display the first 5 rows of daily data
                print(f"\nFirst 5 rows of {symbol} daily data:")
                print(daily_data.head())
            else:
                print(f"Failed to download any data for {symbol}")

    except Exception as e:
        print(f"Error downloading data for {symbol}: {str(e)}")

# Create summary of downloaded files
file_info = []
for symbol in symbols:
    safe_name = symbol.replace('-', '_').replace('.', '_')
    hourly_path = f"data/{safe_name}_hourly.csv"
    daily_path = f"data/{safe_name}_daily.csv"

    if os.path.exists(hourly_path):
        df = pd.read_csv(hourly_path)
        file_info.append({
            'Symbol': symbol,
            'Filename': hourly_path,
            'Rows': len(df),
            'Start Date': df['Datetime'].iloc[0],
            'End Date': df['Datetime'].iloc[-1],
            'Frequency': 'Hourly'
        })
    elif os.path.exists(daily_path):
        df = pd.read_csv(daily_path)
        file_info.append({
            'Symbol': symbol,
            'Filename': daily_path,
            'Rows': len(df),
            'Start Date': df['Date'].iloc[0],
            'End Date': df['Date'].iloc[-1],
            'Frequency': 'Daily'
        })
    else:
        file_info.append({
            'Symbol': symbol,
            'Filename': 'N/A',
            'Rows': 0,
            'Start Date': 'N/A',
            'End Date': 'N/A',
            'Frequency': 'N/A'
        })

# Display file summary
file_summary = pd.DataFrame(file_info)
print("\nFile Summary:")
print(file_summary)

print("\nData collection complete!")

**Possible Implementation Ideas**

In [ ]:
# Perform basic data exploration
def explore_data(data):
    """Generate basic statistics and visualizations of the market data."""
    # Example steps:
    # - Print summary statistics
    # - Plot closing prices over time
    # - Identify any outliers or anomalies

    return None


In [ ]:
# Visualize technical indicators
def visualize_indicators(data):
    """Add and plot technical indicators like SMA, EMA, RSI, etc."""
    # Example steps:
    # - Compute simple and exponential moving averages
    # - Plot with the original closing price
    # - Highlight crossover points

    return None


In [ ]:
# Perform ML Analysis to identify trends and make predictions
def analyze_market_trends(market_data):
    """Apply ML techniques to analyze stock/crypto trends."""
    # Example steps:
    # - Normalize data for better trend detection
    # - Apply time series models (ARIMA, LSTMs, etc.)
    # - Identify market linearity and anomalies
    # - Predict future price movements

    return {}


In [ ]:
# Generate trading signals based on strategy
def generate_trading_signals(data):
    """Define and apply rules to generate buy/sell signals."""
    # Example strategy ideas:
    # - SMA crossover
    # - RSI-based entry/exit
    # - Price breakout detection

    return []


In [ ]:
# Simulate portfolio performance
def simulate_portfolio(data, signals):
    """Backtest the strategy using historical data and generated signals."""
    # Example steps:
    # - Initialize a virtual portfolio
    # - Buy/sell based on signals
    # - Track cumulative returns and drawdowns

    return {}


In [ ]:
# Evaluate strategy performance
def evaluate_strategy(results):
    """Evaluate strategy using key metrics."""
    # Example metrics:
    # - Cumulative returns
    # - Sharpe ratio
    # - Maximum drawdown

    return {}


In [ ]:
# Wrap-up and discussion
def reflect_on_findings():
    """Summarize insights and propose next steps."""
    # Example:
    # - What worked? What didn’t?
    # - How would you improve the strategy?
    # - What other data might help?

    return None
